In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install SentencePiece==0.1.94

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.4 MB/s eta 0:00:00


In [5]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
%cd /content/drive/MyDrive/transformer-drg-style-transfer-master/

/content/drive/.shortcut-targets-by-id/1gvgdEyQQFFN43xnL2_DdI4rUtMI5gnmU/transformer-drg-style-transfer-master


In [7]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration

In [8]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_cls = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment')
# #model_cls.load_state_dict(torch.load('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment/pytorch_model.bin'))
# model_cls.eval()

In [9]:
tokenizer = T5Tokenizer.from_pretrained('t5-base', max_length=70)
model =  T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment', return_dict = True)
model.load_state_dict(torch.load('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment/pytorch_model.bin', map_location=torch.device(device)))
model.eval()

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base', max_length=70)
model =  T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment', return_dict = True)
model.load_state_dict(torch.load('/content/drive/MyDrive/transformer-drg-style-transfer-master/t5_base_yelp_sentiment/pytorch_model.bin', map_location=torch.device(device)))
model.eval()

In [16]:
from tqdm.auto import tqdm

In [ ]:
class YelpDataset(Dataset):
  def __init__(self, tokenizer, data_dir, type_path,  max_len=512, truncation = True):
    self.file_path = '/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/t5/dev.csv'
    
    self.truncation = truncation
    
    self.max_len = max_len
    self.tokenizer = tokenizer
    self.inputs = []
    self.targets = []

    self._build()
  
  def __len__(self):
    return len(self.inputs)
  
  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

    return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}
  
  def _build(self):
    self._buil_examples_from_files(self.file_path)
  
  def _buil_examples_from_files(self, files):
    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()]")
    REPLACE_WITH_SPACE = re.compile("()|(\-)|(\/)")
    df = pd.read_csv('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/t5/dev.csv', on_bad_lines='skip', delimiter='\t', header=None, index_col=None)
    df.dropna(inplace=True)
    df = df.groupby(df[1]).sample(n=500, random_state = 42)
    df[1] = df[1].apply(str)
    # df[0] = df[0].apply(lambda text: REPLACE_NO_SPACE.sub("", text))
    # X = df[0].apply(lambda text : REPLACE_WITH_SPACE.sub("", text))
    X = df[0]
    y = df[1]
    
    
    for index, row in tqdm(enumerate(X)):
      # tokenize inputs
      tokenized_inputs = self.tokenizer.batch_encode_plus([X.iloc[index]], max_length=self.max_len, 
                                                          pad_to_max_length=True, 
                                                          return_tensors="pt", truncation = self.truncation)
      tokenized_targets = self.tokenizer.batch_encode_plus([y.iloc[index]], 
                                                           max_length=2, 
                                                           pad_to_max_length=True, 
                                                           return_tensors="pt", truncation = self.truncation)


       # tokenize targets
    
      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)
     

In [11]:
!nvidia-smi

Sun Apr 23 03:34:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    32W /  70W |   1805MiB / 15360MiB |     32%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
model.cuda()
# model_cls.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

## Attentions

In [10]:
def get_attention_for_encoded_dataset(dataset, batch_size=10):
    """
    returns attention weights in the form of (num_layers, total_data, num_heads, sequence_length, sequence_length)
    """
    all_attn_probs = []
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    it = iter(loader)
    model.to(device)

    with torch.no_grad():
        for batch in tqdm(it):
            batch_attn_probs = []  # To store attention weights for each layer in the current batch
            torch.cuda.empty_cache()
            input_ids = batch.to(device)
            outputs = model(input_ids=input_ids, output_attentions=True, decoder_input_ids=input_ids)
            attn = outputs.cross_attentions

            # Unbatch the attention weights for each layer
            for layer in attn:
                batch_attn_probs.append(layer.detach().cpu().unsqueeze(1))
            batch_attn_probs = torch.cat(batch_attn_probs, dim=1)  # Shape: (batch_size, num_heads, sequence_length, sequence_length)
            all_attn_probs.append(batch_attn_probs)

    # Stack the attention weights across all batches
    all_attn_probs = torch.cat(all_attn_probs, dim=0)  # Shape: (total_data, num_heads, sequence_length, sequence_length)

    # Transpose to get the required shape
    # Shape: (num_layers, sequence_length, total_data, num_heads, sequence_length)

    all_attn_probs = all_attn_probs.transpose(0, 1).transpose(1, 2)  # Shape: (num_layers, num_heads, total_data, sequence_length, sequence_length)
    
    return all_attn_probs


In [11]:
# dataset = YelpDataset(tokenizer, 'aclImdb', 'test',  max_len=72)

df = pd.read_csv('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/t5/dev.csv', on_bad_lines='skip', delimiter='\t', header=None, index_col=None)
df.dropna(inplace=True)
df = df.groupby(df[1]).sample(n=200, random_state = 42)
df[1] = df[1].apply(str)
X = df[0].values.tolist()


input = tokenizer(X, max_length=72, pad_to_max_length=True, 
                                                          return_tensors="pt", truncation = True)
input_ids = input['input_ids'] 

attn =  get_attention_for_encoded_dataset(input_ids, batch_size=10)


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


NameError: ignored

In [ ]:
attn = torch.load('attn.pt')

In [ ]:
attn.size()

torch.Size([12, 12, 200, 72, 72])

In [ ]:
torch.save(attn, 'attn.pt')

In [16]:
import os
import psutil

def clear_ram_cache():
    """
    Clears the RAM cache on a Linux-based system.
    """
    os.system("sync; echo 1 > /proc/sys/vm/drop_caches")
    print("RAM cache cleared.")

clear_ram_cache()

RAM cache cleared.


# removal of top k tokens

In [17]:
def process_sentences(input_sentences, att, remove_special_tokens=True, threshold=0.06):
    """
    This function processes each input sentence by removing the top tokens defined threshold value.
    Each sentence is processed for each head.

    Args:
        input_sentences (list): List of input sentences
        decoding_ids (list): List of encoded input vectors
        att (torch.Tensor): T5 attention tensor of shape (num_layers, num_heads, total_size, seq_length, seq_length)
        remove_special_tokens (bool): Whether to remove special tokens from the output (default: True)
        threshold (float): Percentage of the top indexes to be removed (default: 0.1)

    Returns:
        List of processed sentences, where each sentence is a list of processed strings for each head.
    """
    num_layers, num_heads, total_size, seq_length, _ = att.size()

    # Convert the attention tensor to a NumPy array
    attention_np = att.cpu().detach().numpy()

    # Create an empty list to store the processed sentences
    processed_sentences = [None for j in range((num_heads)*(num_layers))]

    inx = 0
    for i in tqdm(range(num_layers)):
        for j in range(num_heads):
            processed_head = [None for k in range(total_size)]
            for k in range(total_size):
                # Get the encoded input vector for the kth input sentence
                encoded_input_k = input_sentences[k]

                # Get the attention scores for the kth input sentence and jth head
                att_scores = attention_np[i, j, k]
                

                # Get the top indexes to be removed
                topk = int(len(att_scores[0]) * threshold)
                topi = att_scores[0].argsort()[::-1][:topk]


                # Remove the top indexes from the encoded input vector
                # print('------------')
                # print(encoded_input_k)
                
                encoded_input_trimmed = [token_id for idx, token_id in enumerate(encoded_input_k) if idx not in topi]
                #print(encoded_input_trimmed)

                # Convert the encoded input vector back to a string
                decoded_str = tokenizer.decode(encoded_input_trimmed, skip_special_tokens=True)
                #print(decoded_str)
                

                processed_head[k] = decoded_str
            # print('++++++++++++')
            # print(processed_head[:10])
            processed_sentences[inx] = processed_head
            inx+=1
      

    return processed_sentences


In [18]:
# df = pd.read_csv('/content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/t5/dev.csv', on_bad_lines='skip', delimiter='\t', header=None, index_col=None)
# df.dropna(inplace=True)
# df = df.groupby(df[1]).sample(n=500, random_state = 42)
# df[1] = df[1].apply(str)
# X = df[0].values.tolist()


# input = tokenizer(X, max_length=72, pad_to_max_length=True, 
#                                                           return_tensors="pt", truncation = True)
# input_ids = input['input_ids'] 

sen_list = process_sentences(input_ids, attn)

  0%|          | 0/12 [00:00<?, ?it/s]

In [21]:
import pickle

with open("test", "wb") as fp:   #Pickling
  pickle.dump(sen_list, fp)

In [12]:
import pickle
with open('test', 'rb') as f:
    sen_list = pickle.load(f)

In [13]:
def process_multiple_sentences(sen_list, batch_size=10):
    #model_cls.cuda()
    model_inputs = []
    pred_lt = []


    input = tokenizer(sen_list, max_length=72, pad_to_max_length=True, return_tensors="pt", truncation=True)
    input_ids = input['input_ids']

    loader = DataLoader(input_ids, batch_size=batch_size, shuffle=False)
    it = iter(loader)
    model.to(device)
    class_scores = np.zeros((len(sen_list), 2))
    idx = 0
    with torch.no_grad():
        for batch in it:
            torch.cuda.empty_cache()
            batch = batch.to(device)
            outputs = model(input_ids=batch, decoder_input_ids=batch)
            logits = outputs.logits[:, 1]
            probs = torch.softmax(logits, dim=-1)
            scores = probs[:, 1]
            scores_0 = (1 - scores)
            scores = scores.cpu().numpy()
            scores_0 = scores_0.cpu().numpy()
            
            temp_list = np.column_stack((scores_0, scores))
            #print(class_scores.shape, scores.shape, scores_0.shape, temp_list.shape)
            class_scores[idx:idx+batch_size] = temp_list
            idx+=batch_size
    
    
    # Obtain the classification probability of the predicted output

    #print(class_scores)
    return class_scores


In [14]:
def get_block_head(processed_sentence_list, lmbd = 0.1):
    """
    This function calculate classification scores for sentences generated by each head
    and sort them from best to worst.
    score = min(pred) + lmbd / max(pred) + lmbd, lmbd is smoothing param
    pred is list of probability score for each class, for best case pred = [0.5, 0.5] ==> score = 1
    
    it returns sorted list of (Layer, Head, Score)
    """
    scores = {}
    #scores_1 = {}
    for i in tqdm(range(len(processed_sentence_list))): # sentences by each head
        #print(processed_sentence_list[i])
        pred = np.array(process_multiple_sentences(processed_sentence_list[i]))
        scores[i] = np.mean([(min(x[0], x[1])+lmbd)/(max(x[0], x[1])+lmbd) for x in pred])
        #scores_1[i] = np.mean([abs(max(x[0],x[1]) - min(x[0],x[1])) for x in pred])
    temp = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
    #temp1 = sorted(scores_1.items(), key=lambda kv: kv[1], reverse=False)
    score_lt = [(x // 12, x - (12 * (x // 12)),y) for x,y in temp]
    #score1_lt = [(x // 12, x - (12 * (x // 12)),y) for x,y in temp1]
    return score_lt  #score1_lt

In [17]:
scores = get_block_head(sen_list)

  0%|          | 0/144 [00:00<?, ?it/s]

In [18]:
scores

[(6, 0, 0.1886666047780669),
 (8, 3, 0.17818111172669035),
 (6, 11, 0.17782135217180708),
 (6, 6, 0.1768796998756291),
 (11, 3, 0.1765833478632518),
 (1, 7, 0.17653317507348854),
 (8, 10, 0.1745621665480632),
 (5, 0, 0.1740807017821691),
 (4, 0, 0.17360569234366177),
 (10, 10, 0.17328406888563258),
 (6, 9, 0.17286359385375788),
 (6, 7, 0.17272362104082264),
 (2, 7, 0.17172294613856137),
 (1, 11, 0.1716852533925048),
 (11, 9, 0.17160905469217647),
 (6, 1, 0.17099513453388185),
 (5, 10, 0.1702280725273276),
 (7, 7, 0.1695993064579305),
 (11, 10, 0.16950135331922805),
 (4, 1, 0.1692915988612422),
 (9, 8, 0.16895496215591002),
 (9, 2, 0.1684952110633918),
 (11, 7, 0.1684192828801784),
 (7, 11, 0.16823850942629967),
 (4, 9, 0.1680505133021179),
 (5, 4, 0.16782368293738795),
 (3, 6, 0.16750841246488563),
 (5, 7, 0.16748404254579563),
 (7, 10, 0.16703825371470432),
 (11, 2, 0.16696967743442365),
 (10, 0, 0.16690263155451732),
 (7, 8, 0.16689493191398747),
 (1, 4, 0.16687444802620227),
 (1, 3,

In [ ]:
scores

In [ ]:
a = sen_list[0]
b = sen_list[1]
count = 0
for i in range(len(sen_list[0])):
  if a[i] != b[i]:
    print(a[i], b[i], count)
    count += 1